#**Trending Topics Project**
* This project is aimed at developing dashboard visualisation of the Twitter trending topics in different geographical locations.
* The visualizations provide also delves into sentiment analysis of the trending topics to understand whether they are positive or negative.
* The categorization of the topics is necessary to ensure we enhance our visibility in the appropriate trends.
* This project will involve developing a model to read and automate prediction of sentiments from trending topics.

**Tools and Libraries**
* Google Colab: hosting the code, training, testing and evaluating the model.
* Tweepy: packages to access Twitter API and pull tweets.
* Twitter’s API - We shall use Kiboi's Developer Account
* Emojis: package for conversion of emojis into text and vice versa
* Trax / Keras: building a deep neural network.
* Google Cloud Compute Engine: a virtual machine in the cloud to host the script.
* Google Cloud Storage: a bucket in the cloud to store the data.
* Google Cloud Secrets Manager: a place in the cloud to store our secrets and keys.
* Google Data Studio: an interactive dashboard studio (We shall use 247rpwr@gmail.com account)



In [7]:
!pip install emojis
!pip install trax
!pip install tweepy
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 9.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 3.1 MB/s eta 0:00:00


In [8]:
# add to system path
import sys
sys.path.append('/kaggle/working/mysitepackages')

Let's import the necessary libraries and packages

In [9]:
import tweepy                             #package to conveniently access the Twitter API with Python
import json                               #a standard text-based format for representing structured data based on JavaScript object syntax.

import warnings
warnings.filterwarnings("ignore")

#import geocoder
import pandas as pd
import numpy as np

import os 
import time
import emojis


import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import random as rnd                       # pseudo-random number generator

import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings


# import relevant libraries
import trax

# import trax.fastmath.numpy

import trax.fastmath.numpy as np


# import trax.layers
from trax import layers as tl


from trax.supervised import training


In [10]:
    api_key= 'SOmTAIoDSCqOeR3cFaJEt5kOe'
    api_secret= 'dqZL0R6kdaCpIMkDBxYCpBbZ5VHLrO3sNBb83vYef3Tq0lNIvC'
    access_token= '912397733377581056-eZz0fQYAA8uVa3gdapd9s2Xg0ASgwtj'
    access_token_secret= 'nlQ8KJxMswrXcCd5p5wC4B2w2yvxN67aaJPcoCb1C9yhY'

In [11]:
# authorize api handshake
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

## Downlaod training data

In [12]:
# downloads sample twitter dataset. uncomment the line below if running on a local machine.
nltk.download('twitter_samples') 

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [13]:
# select the set of positive and negative tweets
nltk_positive_tweets = twitter_samples.strings('positive_tweets.json')
nltk_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [14]:
import pandas as pd

file_path = '/kaggle/input/data-tweets/Training Tweets.csv'
tweets = pd.read_csv(file_path, encoding = "ISO-8859-1", names = ["target", "ids", "date", "flag", "user", "text"])
tweets.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   ids     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [16]:
tweets['target'].unique()

array([0, 4])

In [17]:
fmap = {4:1, 0:0}

tweets['target'] = tweets['target'].map(fmap)

tweets['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

In [18]:
pos_tweets = list(tweets[tweets['target'] == 1]['text'])
all_positive_tweets = nltk_positive_tweets + pos_tweets

neg_tweets = list(tweets[tweets['target'] == 0]['text'])
all_negative_tweets = nltk_negative_tweets + neg_tweets

In [19]:
print('Number of positive tweets: ', len(all_positive_tweets))
print('Number of negative tweets: ', len(all_negative_tweets))

Number of positive tweets:  805000
Number of negative tweets:  805000


In [20]:
# print positive in greeen
print('\033[92m' + all_positive_tweets[rnd.randint(0,805000)])

&quot;the boys are back!&quot; that pink cake was.....weird 


In [21]:
# print negative in red
print('\033[91m' + all_negative_tweets[rnd.randint(0,805000)])

I want a yo-yo 


In [22]:
percentage = (len(all_positive_tweets[:724500]) / len(all_positive_tweets)) * 100
print("percentage used in training:", percentage)

percentage = (len(all_positive_tweets[724500:]) / len(all_positive_tweets)) * 100
print("percentage used for validation:", percentage)

percentage used in training: 90.0
percentage used for validation: 10.0


In [23]:
# Split positive set into validation and training
val_pos   = all_positive_tweets[724500:] # generating validation set for positive tweets
train_pos  = all_positive_tweets[:724500]# generating training set for positive tweets

# Split negative set into validation and training
val_neg   = all_negative_tweets[724500:] # generating validation set for negative tweets
train_neg  = all_negative_tweets[:724500] # generating training set for nagative tweets

# Combine training data into one set
train_x = train_pos + train_neg 

# Combine validation data into one set
val_x  = val_pos + val_neg

# Set the labels for the training set (1 for positive, 0 for negative)
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))

# Set the labels for the validation set (1 for positive, 0 for negative)
val_y  = np.append(np.ones(len(val_pos)), np.zeros(len(val_neg)))

print(f"Length of train_x:  {len(train_x)}")
print(f"Length of val_x:  {len(val_x)}")

Length of train_x:  1449000
Length of val_x:  161000


**Pre-process Tweets**

In [24]:
# Our selected sample. Complex enough to exemplify each step
tweet = all_positive_tweets[100]
print(tweet)

print(' ')

tweet2 = all_negative_tweets[90]
print(tweet2)

@metalgear_jp @Kojima_Hideo I want you're T-shirts ! They are so cool ! :D
 
Wft.. can't watch the awesome replay!! :-( https://t.co/ChzrqtelPh


In [25]:
# download the stopwords from NLTK
nltk.download('stopwords')

#Import the english stop words list from NLTK
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
import re
import string
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import emojis

def process_tweet(tweet):
    # Remove old style retweet text "RT"
    new_tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # Decode emojis to text descriptions
    new_tweet = emojis.decode(new_tweet)

    # Remove hyperlinks
    new_tweet = re.sub(r'http\S+|www\.[^\s]+', '', new_tweet)
    
    # Remove hashtags and underscores
    new_tweet = re.sub(r'#[^\s]+|_', '', new_tweet)

    # Remove all numbers
    new_tweet = re.sub(r'\d+', '', new_tweet)

    # Remove usernames
    new_tweet = re.sub(r'@[^\s]+', '', new_tweet)
    
    # Remove punctuation even in the middle of a string "in.the.middle"
    new_tweet = re.sub(r'[^\w\s]', ' ', new_tweet)
    
    # Convert text to lowercase
    new_tweet = new_tweet.lower()

    # Instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

    # Tokenize tweets
    tweet_tokens = tokenizer.tokenize(new_tweet)

    stopwords_english = set(stopwords.words('english'))
    
    # Remove stopwords and punctuation
    tweets_clean = [word for word in tweet_tokens if word not in stopwords_english and word not in string.punctuation]

    # Instantiate stemming class
    stemmer = PorterStemmer() 

    # Create an empty list to store the stems
    tweets_stem = [stemmer.stem(word) for word in tweets_clean]  # Stemming word and append to the list
    
    return tweets_stem


In [27]:
tweet = all_positive_tweets[500]
print(f'Original tweet:     {tweet}')

processed_tweet = process_tweet(tweet)
print(f'Processed tweet:     {processed_tweet}')

Original tweet:     Dang that is some rad @AbzuGame #fanart! :D https://t.co/bI8k8tb9ht
Processed tweet:     ['dang', 'rad']


**Build the Vocabulary**

In [28]:
Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

In [29]:
# Build the vocabulary

# Include special tokens 
# started with pad, end of line and unk tokens

# Note that we build vocab using training data
for tweet in train_x: 
    processed_tweet = process_tweet(tweet)
    for word in processed_tweet:
        if word not in Vocab: 
            Vocab[word] = len(Vocab)
    
print("Total words in vocab are", len(Vocab), '\n')

first_10 = {k: Vocab[k] for k in sorted(Vocab.keys())[:10]}
display(first_10)

Total words in vocab are 193968 



{'__</e>__': 1,
 '__PAD__': 0,
 '__UNK__': 2,
 'a': 54514,
 'aa': 3406,
 'aaa': 9203,
 'aaacattyisamaz': 186841,
 'aaaccee': 63539,
 'aaacckk': 166602,
 'aaach': 105154}

In [30]:
json.dump(Vocab, open( "Vocab.json", 'w' ) )

**Build a Tensor from a Tweet**

In [31]:
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
        
    '''  

    # Process the tweet into a list of words
    # where only important words are kept (stop words removed)
    word_l = process_tweet(tweet)
    
    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)
        
    # Initialize the list that will contain the unique integer IDs of each word
    tensor_l = []
    
    # Get the unique integer ID of the __UNK__ token
    unk_ID = vocab_dict[unk_token]
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
        
    # for each word in the list:
    for word in word_l:
        
        # Get the unique integer ID.
        # If the word doesn't exist in the vocab dictionary,
        # use the unique ID for __UNK__ instead.
        word_ID = vocab_dict.get(word, unk_ID)

        
        # Append the unique integer ID to the tensor list.
        tensor_l.append(word_ID) 
    
    return tensor_l

In [32]:
print("Actual tweet is\n", val_pos[1])
print("\nTensor of tweet:\n", tweet_to_tensor(val_pos[1], vocab_dict=Vocab))

Actual tweet is
 4 hours and counting 

Tensor of tweet:
 [520, 1860]


**Build a Batch Generator**

In [33]:
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
        data_pos - Set of posstive examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be even
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - An array specifying the importance of each example
        
    '''     

    # make sure the batch size is an even number
    # to allow an equal number of positive and negative samples
    assert batch_size % 2 == 0
    
    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2
    
    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
    
    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    
    # shuffle lines if shuffle is set to True
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)
        
    stop = False
    
    # Loop indefinitely
    while not stop:  
        
        # create a batch with positive and negative examples
        batch = []
        
        # First part: Pack n_to_take positive examples
        
        # Start from pos_index and increment i up to n_to_take
        for i in range(n_to_take):
                    
            # If the positive index goes past the positive dataset lenght,
            if pos_index >= len_data_pos: 
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                
                # If user wants to keep re-using the data, reset the index
                pos_index = 0
                
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)
                    
            # get the tweet as pos_index
            tweet = data_pos[pos_index_lines[pos_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment pos_index by one
            pos_index = pos_index + 1


        # Second part: Pack n_to_take negative examples
    
        # Using the same batch list, start from neg_index and increment i up to n_to_take
        for i in range(n_to_take):
            
            # If the negative index goes past the negative dataset length,
            if neg_index >= len_data_neg:
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                    
                # If user wants to keep re-using the data, reset the index
                neg_index = 0
                
                if shuffle:
                    # Shuffle the index of the negative sample
                    rnd.shuffle(neg_index_lines)
            # get the tweet as neg_index
            tweet = data_neg[neg_index_lines[neg_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment neg_index by one
            neg_index = neg_index + 1

        if stop:
            break;

        # Update the start index for positive data 
        # so that it's n_to_take positions after the current pos_index
        pos_index += n_to_take
        
        # Update the start index for negative data 
        # so that it's n_to_take positions after the current neg_index
        neg_index += n_to_take
        
        # Get the max tweet length (the length of the longest tweet) 
        # (you will pad all shorter tweets to have this length)
        max_len = max([len(t) for t in batch]) 
        
        
        # Initialize the input_l, which will 
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter tweets with zeros
        for tensor in batch:

            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)
            
            # Generate a list of zeros, with length n_pad
            pad_l = [0] * n_pad
            
            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor +  pad_l

            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)

        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = np.array(tensor_pad_l)
  
        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos = [1] * n_to_take
        
        # Generate the list of targets for the negative examples (a list of zeros)
        # The length is the number of negative examples in the batch
        target_neg = [0] * n_to_take
        
        # Concatenate the positve and negative targets
        target_l = target_pos + target_neg
        
        # Convert the target list into a numpy array
        targets = np.array(target_l)

        # Example weights: Treat all examples equally importantly.It should return an np.array. Hint: Use np.ones_like()
        example_weights = np.ones_like(targets)
        
        # note we use yield and not return
        yield inputs, targets, example_weights

In [34]:
# Set the random number generator for the shuffle procedure
rnd.seed(42) 

# Create the training data generator
def train_generator(batch_size, shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def val_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def test_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, False, Vocab, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[  585   134  1168   134   222     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [  579    66  4445   886  5054  8542     0     0     0     0     0     0
      0     0     0     0     0     0]
 [ 1215 10389   555  1651    21 44576  1269 16032  4103  2947 21614 14673
   5304  2712  1414   202    56  5484]
 [ 7978   833  3093   316   328   404  1965 16067   203     0     0     0
      0     0     0     0     0     0]]
Targets: [1 1 0 0]
Example Weights: [1 1 1 1]


**Build a Model**

In [35]:
def classifier(vocab_size=len(Vocab), embedding_dim=256, output_dim=2, mode='train'):
        

    # create embedding layer
    embed_layer = tl.Embedding(
        vocab_size=vocab_size, # Size of the vocabulary
        d_feature=embedding_dim)  # Embedding dimension
    
    # Create a mean layer, to create an "average" word embedding
    mean_layer = tl.Mean(axis=1)
    
    # Create a dense layer, one unit for each output
    dense_output_layer = tl.Dense(n_units = output_dim)

    # Create the log softmax layer (no parameters needed)
    log_softmax_layer = tl.LogSoftmax()
    
    # Use tl.Serial to combine all layers
    # and create the classifier
    # of type trax.layers.combinators.Serial
    model = tl.Serial(
      embed_layer, # embedding layer
      mean_layer, # mean layer
      dense_output_layer, # dense output layer 
      log_softmax_layer # log softmax layer
    )

    # return the model of type
    return model

In [36]:
tmp_model = classifier()

print(type(tmp_model))
display(tmp_model)

<class 'trax.layers.combinators.Serial'>


Serial[
  Embedding_193968_256
  Mean
  Dense_2
  LogSoftmax
]

**Train the Model**

In [37]:
batch_size = 16
rnd.seed(42)

train_task = training.TrainTask(
    labeled_data=train_generator(batch_size=batch_size, shuffle=True),
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.0001),
    n_steps_per_checkpoint=100,
)

eval_task = training.EvalTask(
    labeled_data=val_generator(batch_size=batch_size, shuffle=True),
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
)

model = classifier()

In [38]:
! pwd

/kaggle/working


In [39]:
output_dir = '~/kaggle/working'
output_dir_expand = os.path.expanduser(output_dir)
print(output_dir_expand)

/root/kaggle/working


In [40]:
def train_model(classifier, train_task, eval_task, n_steps, output_dir):
    '''
    Input: 
        classifier - the model you are building
        train_task - Training task
        eval_task - Evaluation task
        n_steps - the evaluation steps
        output_dir - folder to save your files
    Output:
        trainer -  trax trainer
    '''

    training_loop = training.Loop(
                                classifier, # The learning model
                                train_task, # The training task
                                eval_tasks = eval_task, # The evaluation task
                                output_dir = output_dir) # The output directory

    training_loop.run(n_steps = n_steps)

    # Return the training_loop, since it has the model.
    return training_loop

In [41]:
training_loop = train_model(model, train_task, eval_task, 100, output_dir_expand)


Step      1: Total number of trainable weights: 49656322
Step      1: Ran 1 train steps in 15.95 secs
Step      1: train CrossEntropyLoss |  0.68956083
Step      1: eval  CrossEntropyLoss |  0.69308597
Step      1: eval          Accuracy |  0.50000000

Step    100: Ran 99 train steps in 81.68 secs
Step    100: train CrossEntropyLoss |  0.69269347
Step    100: eval  CrossEntropyLoss |  0.69442624
Step    100: eval          Accuracy |  0.56250000


**Testing the Model**

In [42]:
# Create a generator object
tmp_train_generator = train_generator(16)

# get one batch
tmp_batch = next(tmp_train_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

print(f"The batch is a tuple of length {len(tmp_batch)} because position 0 contains the tweets, and position 1 contains the targets.") 
print(f"The shape of the tweet tensors is {tmp_inputs.shape} (num of examples, length of tweet tensors)")
print(f"The shape of the labels is {tmp_targets.shape}, which is the batch size.")
print(f"The shape of the example_weights is {tmp_example_weights.shape}, which is the same as inputs/targets size.")

The batch is a tuple of length 3 because position 0 contains the tweets, and position 1 contains the targets.
The shape of the tweet tensors is (16, 13) (num of examples, length of tweet tensors)
The shape of the labels is (16,), which is the batch size.
The shape of the example_weights is (16,), which is the same as inputs/targets size.


In [43]:
# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)
print(f"The prediction shape is {tmp_pred.shape}, num of tensor_tweets as rows")
print("Column 0 is the probability of a negative sentiment (class 0)")
print("Column 1 is the probability of a positive sentiment (class 1)")
print()
print("View the prediction array")
tmp_pred

The prediction shape is (16, 2), num of tensor_tweets as rows
Column 0 is the probability of a negative sentiment (class 0)
Column 1 is the probability of a positive sentiment (class 1)

View the prediction array


Array([[-0.68743336, -0.6988939 ],
       [-0.7301458 , -0.6574688 ],
       [-0.7090331 , -0.6775097 ],
       [-0.69337684, -0.69291764],
       [-0.70025253, -0.68609196],
       [-0.69587654, -0.6904253 ],
       [-0.71288025, -0.673796  ],
       [-0.7157861 , -0.67100936],
       [-0.69728625, -0.68902516],
       [-0.69233763, -0.6939574 ],
       [-0.69005144, -0.69625264],
       [-0.6969838 , -0.68932533],
       [-0.7143808 , -0.672355  ],
       [-0.6999406 , -0.6863996 ],
       [-0.69180053, -0.69449574],
       [-0.6597025 , -0.7277493 ]], dtype=float32)

In [44]:
# turn probabilites into category predictions
tmp_is_positive = tmp_pred[:,1] > tmp_pred[:,0]
for i, p in enumerate(tmp_is_positive):
    print(f"Neg log prob {tmp_pred[i,0]:.4f}\tPos log prob {tmp_pred[i,1]:.4f}\t is positive? {p}\t actual {tmp_targets[i]}")

Neg log prob -0.6874	Pos log prob -0.6989	 is positive? False	 actual 1
Neg log prob -0.7301	Pos log prob -0.6575	 is positive? True	 actual 1
Neg log prob -0.7090	Pos log prob -0.6775	 is positive? True	 actual 1
Neg log prob -0.6934	Pos log prob -0.6929	 is positive? True	 actual 1
Neg log prob -0.7003	Pos log prob -0.6861	 is positive? True	 actual 1
Neg log prob -0.6959	Pos log prob -0.6904	 is positive? True	 actual 1
Neg log prob -0.7129	Pos log prob -0.6738	 is positive? True	 actual 1
Neg log prob -0.7158	Pos log prob -0.6710	 is positive? True	 actual 1
Neg log prob -0.6973	Pos log prob -0.6890	 is positive? True	 actual 0
Neg log prob -0.6923	Pos log prob -0.6940	 is positive? False	 actual 0
Neg log prob -0.6901	Pos log prob -0.6963	 is positive? False	 actual 0
Neg log prob -0.6970	Pos log prob -0.6893	 is positive? True	 actual 0
Neg log prob -0.7144	Pos log prob -0.6724	 is positive? True	 actual 0
Neg log prob -0.6999	Pos log prob -0.6864	 is positive? True	 actual 0
Neg

**Build Accuracy Function & Test on One Batch of Data**

In [45]:
def compute_accuracy(preds, y, y_weights):
    """
    Input: 
        preds: a tensor of shape (dim_batch, output_dim) 
        y: a tensor of shape (dim_batch, output_dim) with the true labels
        y_weights: a n.ndarray with the a weight for each example
    Output: 
        accuracy: a float between 0-1 
        weighted_num_correct (np.float32): Sum of the weighted correct predictions
        sum_weights (np.float32): Sum of the weights
    """

    # Create an array of booleans, 
    # True if the probability of positive sentiment is greater than
    # the probability of negative sentiment
    # else False
    is_pos =  preds[:,1] > preds[:,0]

    # convert the array of booleans into an array of np.int32
    is_pos_int = np.array(is_pos, dtype = np.int32)
    
    # compare the array of predictions (as int32) with the target (labels) of type int32
    correct = is_pos_int == y

    # Count the sum of the weights.
    sum_weights = np.sum(y_weights)
    
    # convert the array of correct predictions (boolean) into an arrayof np.float32
    correct_float = np.array(correct, dtype = np.float32)
    
    # Multiply each prediction with its corresponding weight.
    weighted_correct_float = correct_float * y_weights

    # Sum up the weighted correct predictions (of type np.float32), to go in the
    # denominator.
    weighted_num_correct = np.sum(weighted_correct_float)
 
    # Divide the number of weighted correct predictions by the sum of the
    # weights.
    accuracy = sum_weights / weighted_num_correct 

    return accuracy, weighted_num_correct, sum_weights

In [46]:
# test your function
tmp_val_generator = val_generator(64)

# get one batch
tmp_batch = next(tmp_val_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)

tmp_acc, tmp_num_correct, tmp_num_predictions = compute_accuracy(preds=tmp_pred, y=tmp_targets, y_weights=tmp_example_weights)

print(f"Model's prediction accuracy on a single training batch is: {100 * tmp_acc}%")
print(f"Weighted number of correct predictions {tmp_num_correct}; weighted number of total observations predicted {tmp_num_predictions}")

Model's prediction accuracy on a single training batch is: 188.23529052734375%
Weighted number of correct predictions 34.0; weighted number of total observations predicted 64


**Model Evaluation**

In [47]:
def test_model(generator, model):
    '''
    Input: 
        generator: an iterator instance that provides batches of inputs and targets
        model: a model instance 
    Output: 
        accuracy: float corresponding to the accuracy
    '''
    
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0

    for batch in generator: 
        
        # Retrieve the inputs from the batch
        inputs = batch[0]
        
        # Retrieve the targets (actual labels) from the batch
        targets = batch[1]
        
        # Retrieve the example weight.
        example_weight = batch[2]

        # Make predictions using the inputs
        pred = model(inputs)
        
        # Calculate accuracy for the batch by comparing its predictions and targets
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(preds=pred, y=targets, y_weights=example_weight)
        
        # Update the total number of correct predictions
        # by adding the number of correct predictions from this batch
        total_num_correct += batch_num_correct
        
        # Update the total number of predictions 
        # by adding the number of predictions made for the batch
        total_num_pred += batch_num_pred

    # Calculate accuracy over all examples
    accuracy = total_num_correct / total_num_pred

    return accuracy

In [48]:
# testing the accuracy of your model: this takes around 20 seconds
model = training_loop.eval_model

accuracy = test_model(test_generator(16), model)

print(f'The accuracy of your model on the validation set is {accuracy:.4f}', )

The accuracy of your model on the validation set is 0.5387


#**Getting Live Twitter Data**

In [49]:
# kenya_woeid = 23424863
usa_woeid = 2442047

# Retrieve top 50 trending topics for Kenya
trending_topics = api.get_place_trends(id=usa_woeid, exclude="hashtags")

topic_list = []

# Loop through each trending topic and create a list
for trend in trending_topics[0]['trends']:
    topic = trend['name']
    topic_list.append(topic)
    print(topic)
auth_params = {
    'app_key': 'SOmTAIoDSCqOeR3cFaJEt5kOe',
    'app_secret': 'dqZL0R6kdaCpIMkDBxYCpBbZ5VHLrO3sNBb83vYef3Tq0lNIvC',
    'oauth_token': '912397733377581056-eZz0fQYAA8uVa3gdapd9s2Xg0ASgwtj',
    'oauth_token_secret': 'nlQ8KJxMswrXcCd5p5wC4B2w2yvxN67aaJPcoCb1C9yhY',
}

$COIN
PETA
Jim Jordan
Trey Lance
New World Order
The SEC
George Santos
Prince Harry
Garland
UFOs
Apple Music
Twitch
Saudi
PGA Tour
Kick
Chris Christie
Provorov
Alcaraz
Elly De La Cruz
Hart
Flyers
Zion
Jay Monahan
Canada
D-Day
Tsitsipas
Byron
Normandy
Rory
Tiger
Briere
Barr
Order 66
Provy
Scoot
SPLC
Coinbase
GMX Аirdrop
Manoah
Gavrikov
Florida Complex League
Baby Gronk
Griz
Phil
Arnold


In [69]:
# !pip install --upgrade tweepy

In [51]:
program_start = time.time()            # time the program

max_tweets = 50        # number of tweets to grab for each topic
n = 50                 # top n number of retweets to keep

#assert (max_tweets * len(df) * 50) <= 100000 , 'Request Limit Assert Error: Too many tweets! \n \n Twitter has a 24 hour limit of 100,000 requests \n \n Try changing "max_tweets"'    
    
columns = ['username', 'user_location', 'retweetcount', 'favorites', 'text',   # set up columns for dataframes
           'hashtags', 'emojis', 'trending_topic', 'search_type']          

recent_tweets_data = pd.DataFrame(columns = columns)                           # create empty dataframe    
    

for k in range(len(trending_topics[0]['trends'])):                # loop though each trending topic for each town
    
    run_start = time.time()                                       # start time of the run for each topic   

    topic = trending_topics[0]['trends'][k]['name']               # the trending topic or hashtag (1 out of 50)  
                                                        
                                                                            # grab tweets with Cursor
    tweets = tweepy.Cursor(api.search_tweets, q = topic,                           # search for each trending topic                                 
                     lang="en", result_type = 'recent' ,                    # most recent tweets in english
                      tweet_mode = 'extended').items(max_tweets)            # longer tweets and grab max_tweets number of tweets
    
    tweet_list = [tweet for tweet in tweets]                                # create list of tweets
                
    recent_tweets_topic = pd.DataFrame(columns = columns)         # create dataframe to put in current top tweets for this town and trending topic
        
    for tweet in tweet_list:                                      # loop through each tweet that was grabbed
        
        username = tweet.user.screen_name              # store username
        user_location = tweet.user.location            # store location of user
        retweetcount = tweet.retweet_count             # store retweet count
        favorites = tweet.favorite_count               # store favorite count
        hashtags = tweet.entities['hashtags']          # store hashtags
        search_type = 'recent'
    
        try:                              
            text = tweet.retweeted_status.full_text    # store text if it's a retweet
        
        except AttributeError: 
            text = tweet.full_text                     # store text if it's a regular tweet
            
        emoji = list(emojis.get(text))                      # get the emojis
        
        curr_tweet = [username, user_location, retweetcount, favorites, text,            # store current tweet's data in a list soon to be a row
                                hashtags, emoji, topic, search_type]                             
        
        recent_tweets_topic.loc[len(recent_tweets_topic)] = curr_tweet                   # add current tweet data to dataframe for town and topic
                            
    recent_tweets_topic.sort_values(by=['retweetcount', 'favorites'], inplace = True, ascending = False)        # sort the retweet values highest first
                            
    top_n = recent_tweets_topic[0:n]                                                     # keep only the top n
                            
    recent_tweets_data = pd.concat([recent_tweets_data, top_n], ignore_index = True, sort = False)              # concatenate top n to final dataframe
    
    run_end = time.time()                                                              # stop the time for the run
    run_duration = round((run_end - run_start)/60, 2)                                  # calculate duration
    print(f'Time for {topic} : {run_duration} mins')                                   # print duration       
    

program_end = time.time()                                                               # stop the program time
program_duration = round(((program_end - program_start) / 60), 2)                       # calculate duration
print(f'\n \n ************************************************** \n \n TOTAL TIME TO GRAB TWEETS : \t {program_duration} mins')    # print duration as total time
print('\n **************************************************')    


Time for $COIN : 0.02 mins
Time for PETA : 0.02 mins
Time for Jim Jordan : 0.02 mins
Time for Trey Lance : 0.02 mins
Time for New World Order : 0.03 mins
Time for The SEC : 0.01 mins
Time for George Santos : 0.02 mins
Time for Prince Harry : 0.02 mins
Time for Garland : 0.02 mins
Time for UFOs : 0.02 mins
Time for Apple Music : 0.03 mins
Time for Twitch : 0.02 mins
Time for Saudi : 0.02 mins
Time for PGA Tour : 0.02 mins
Time for Kick : 0.02 mins
Time for Chris Christie : 0.02 mins
Time for Provorov : 0.02 mins
Time for Alcaraz : 0.03 mins
Time for Elly De La Cruz : 0.02 mins
Time for Hart : 0.02 mins
Time for Flyers : 0.02 mins
Time for Zion : 0.03 mins
Time for Jay Monahan : 0.02 mins
Time for Canada : 0.02 mins
Time for D-Day : 0.02 mins
Time for Tsitsipas : 0.02 mins
Time for Byron : 0.02 mins
Time for Normandy : 0.02 mins
Time for Rory : 0.02 mins
Time for Tiger : 0.02 mins
Time for Briere : 0.02 mins
Time for Barr : 0.02 mins
Time for Order 66 : 0.02 mins
Time for Provy : 0.01 mi

In [52]:
recent_tweets_data.head()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
0,asiyayume1020,JohnnysとTVに出ている人が大好きで憧れる人です！,130918,0,"🎊 READY, SET, GO, THE $LOYAL AIRDROP IS LIVE!\...","[{'text': 'crypto', 'indices': [103, 110]}, {'...","[🔗, 🎁, 🎊]",$COIN,recent
1,asiyayume1020,JohnnysとTVに出ている人が大好きで憧れる人です！,102619,0,LMAO? I JUST CLAIMED $1300 WORTH OF $LOYAL TOK...,[],"[🔗, ‼️]",$COIN,recent
2,phoebewins,wins please.,833,0,.@jimcramer: If you have your money with @bina...,[],[],$COIN,recent
3,SDS_Sunlight,,309,0,COINBASE SIMULTANEOUSLY RECEIVES SHOW CAUSE OR...,[],[],$COIN,recent
4,RuReadyforsome,Canada,120,0,"So in SEC's complaint vs Coinbase, it states t...",[],[],$COIN,recent


In [53]:
len(recent_tweets_data)

2099

In [54]:
program_start = time.time()            # time the program

max_tweets = 300        # number of tweets to grab for each topic
n = 300                 # top n number of retweets to keep

#assert (max_tweets * len(df) * 50) <= 100000 , 'Request Limit Assert Error: Too many tweets! \n \n Twitter has a 24 hour limit of 100,000 requests \n \n Try changing "max_tweets"'


columns = ['username', 'user_location', 'retweetcount', 'favorites', 'text',   # set up columns for dataframes
           'hashtags', 'emojis', 'trending_topic', 'search_type']          

pop_tweets_data = pd.DataFrame(columns = columns)                 # create empty dataframe
    

for q in range(len(trending_topics[0]['trends'])):                # loop though each trending topic for each town

    topic = trending_topics[0]['trends'][q]['name']               # the trending topic or hashtag (1 out of 50)  
                                                                           
                                                                            # grab tweets with Cursor
    tweets = tweepy.Cursor(api.search_tweets, q = topic,                           # search for each trending topic                                 
                     lang="en", result_type = 'popular' ,                   # most recent tweets in english
                      tweet_mode = 'extended').items(max_tweets)            # longer tweets and grab max_tweets number of tweets
    
    tweet_list = [tweet for tweet in tweets]                                # create list of tweets
                
    pop_tweets_topic = pd.DataFrame(columns = columns)         # create dataframe to put in current top tweets for this town and trending topic
        
    for tweet in tweet_list:                                   # loop through each tweet that was grabbed
        
        username = tweet.user.screen_name              # store username
        user_location = tweet.user.location            # store location of user
        retweetcount = tweet.retweet_count             # store retweet count
        favorites = tweet.favorite_count               # store favorite count
        hashtags = tweet.entities['hashtags']          # store hashtags
        search_type = 'popular'
        
        try:                              
            text = tweet.retweeted_status.full_text    # store text if it's a retweet
        
        except AttributeError: 
            text = tweet.full_text                     # store text if it's a regular tweet
            
        emoji = list(emojis.get(text))                      # get the emojis    
        
        curr_tweet = [username, user_location, retweetcount, favorites, text,      # store current tweet's data in a list soon to be a row
                                hashtags, emoji, topic, search_type]                             
        
        pop_tweets_topic.loc[len(pop_tweets_topic)] = curr_tweet                   # add current tweet data to dataframe for town and topic
                            
    pop_tweets_topic.sort_values(by=['retweetcount', 'favorites'], inplace = True, ascending = False)        # sort the retweet values highest first
                            
    top_n = pop_tweets_topic[0:n]                                                                  # keep only the top n
                            
    pop_tweets_data = pd.concat([pop_tweets_data, top_n], ignore_index = True, sort = False)       # concatenate top n to final dataframe
            
    
program_end = time.time()                                                               # stop the program time
program_duration = round(((program_end - program_start) / 60), 2)                       # calculate duration
print(f'\n \n ************************************************** \n \n TOTAL TIME TO GRAB TWEETS : \t {program_duration} mins')    # print duration as total time
print('\n **************************************************')    


 
 ************************************************** 
 
 TOTAL TIME TO GRAB TWEETS : 	 0.55 mins

 **************************************************


In [55]:
# pop_tweets_data.head()

In [56]:
len(pop_tweets_data)

1119

In [57]:
pop_tweets_data['trending_topic'].value_counts()

Canada                    62
Saudi                     61
The SEC                   56
Twitch                    56
Kick                      43
PETA                      39
Apple Music               39
Garland                   36
Coinbase                  35
Hart                      34
Phil                      33
Prince Harry              31
Normandy                  31
PGA Tour                  31
Flyers                    30
Alcaraz                   29
D-Day                     29
Rory                      29
Chris Christie            29
Tiger                     28
Elly De La Cruz           27
Barr                      27
UFOs                      27
New World Order           25
Jim Jordan                25
Arnold                    25
$COIN                     22
George Santos             22
Byron                     21
SPLC                      18
Tsitsipas                 15
Scoot                     15
Manoah                    15
Jay Monahan               14
Provorov      

In [58]:
tweets_data = pd.concat([recent_tweets_data, pop_tweets_data], ignore_index=True)

tweets_data.head()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
0,asiyayume1020,JohnnysとTVに出ている人が大好きで憧れる人です！,130918,0,"🎊 READY, SET, GO, THE $LOYAL AIRDROP IS LIVE!\...","[{'text': 'crypto', 'indices': [103, 110]}, {'...","[🔗, 🎁, 🎊]",$COIN,recent
1,asiyayume1020,JohnnysとTVに出ている人が大好きで憧れる人です！,102619,0,LMAO? I JUST CLAIMED $1300 WORTH OF $LOYAL TOK...,[],"[🔗, ‼️]",$COIN,recent
2,phoebewins,wins please.,833,0,.@jimcramer: If you have your money with @bina...,[],[],$COIN,recent
3,SDS_Sunlight,,309,0,COINBASE SIMULTANEOUSLY RECEIVES SHOW CAUSE OR...,[],[],$COIN,recent
4,RuReadyforsome,Canada,120,0,"So in SEC's complaint vs Coinbase, it states t...",[],[],$COIN,recent


In [59]:
# tweets_data.tail()

In [60]:
len(tweets_data)

3218

**Use Models on Live Twitter Data**

In [61]:
# this is used to predict on your own sentnece
def predict(sentence):
    inputs = np.array(tweet_to_tensor(sentence, vocab_dict=Vocab))
    
    # Batch size 1, add dimension for batch, to work with the model
    inputs = inputs[None, :]  
    
    # predict with the model
    preds_probs = model(inputs)
    
    # Turn probabilities into categories
    preds = int(preds_probs[0, 1] > preds_probs[0, 0])
    
    sentiment = "negative"
    if preds == 1:
        sentiment = 'positive'

    return sentiment, round(float(preds_probs[0, 0]),4), round(float(preds_probs[0, 1]),4)

In [62]:
for i in range(len(tweets_data)):

    tweets_data.loc[i, 'sentiment'], tweets_data.loc[i, 'neg_prob'], tweets_data.loc[i, 'pos_prob'] = predict(tweets_data['text'].iloc[i])

In [63]:
tweets_data.sample(50)

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type,sentiment,neg_prob,pos_prob
1601,zenskywaIker,• 22 • he/him • infp •,0,0,@ArmskillerV2 there were 10 000 jedi during th...,[],[],Order 66,recent,positive,-0.7230,-0.6642
2976,NoLayingUp,"Neptune Beach, FL",86,2314,Rory:\n\n-Bogey on 5 with a wedge\n-Bogey on 7...,[],[],Rory,popular,positive,-0.7517,-0.6378
931,oledicktwist,,0,0,"If Briere got THAT for Provorov, what the fuck...",[],[],Hart,recent,negative,-0.6450,-0.7437
1723,BrooksPepper1,,0,0,@Tyler2ONeil @Moms4Liberty @DefendingEd @Right...,[],[],SPLC,recent,negative,-0.6799,-0.7066
319,adimatoulare,,533,0,Prince Harry launched an unprecedented politic...,[],[],Prince Harry,recent,negative,-0.6646,-0.7226
728,enrique28546,"Jacksonville, NC",9,0,"Reminder: As governor of New Jersey, Chris Chr...",[],[],Chris Christie,recent,negative,-0.6753,-0.7114
1883,fairwaygail,"Ontario, Canada",15,0,BREAKING: #Bluejays announce they have optione...,"[{'text': 'Bluejays', 'indices': [31, 40]}]",[],Florida Complex League,recent,negative,-0.6855,-0.7009
2710,barstoolsports,America,16,311,Reds Call Up Baseball's Top Prospect And One M...,[],[],Elly De La Cruz,popular,positive,-0.7029,-0.6835
493,hashirama102,"📍Xharlotte, NC 🕊🦆🦅",2,0,Niggas Hate On Meh ! Cus I Ain’t In They Leagu...,[],[],Apple Music,recent,positive,-0.7027,-0.6837
1049,4pfmegavirgin,pain,0,0,zion looking slim https://t.co/WA4v7npWCt,[],[],Zion,recent,negative,-0.6300,-0.7605


In [64]:
# max_pos = max(tweets_data['pos_prob'])

# pos_tweets  = tweets_data.loc[tweets_data['pos_prob'] == max_pos][['text']]

# for p in pos_tweets['text']:
#     print(p) 

In [65]:
# max_neg = max(tweets_data['neg_prob'])

# neg_tweets  = tweets_data.loc[tweets_data['neg_prob'] == max_neg][['text']]

# for n in neg_tweets['text']:
#     print(n) 


In [72]:
difficult = "Shit! I have a no money."

predict(difficult)

('negative', -0.6043, -0.7906)

In [67]:
pred_positive_tweets = tweets_data.sort_values(by=['pos_prob'], inplace = False, ascending = False)       
                            
top_positive = pred_positive_tweets[0:25]     

top_positive

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type,sentiment,neg_prob,pos_prob
1096,SportsEsquire,"Charlottesville, VA",0,0,Jay Monahan right now: https://t.co/LDtAYm5UsP,[],[],Jay Monahan,recent,positive,-0.7785,-0.6145
981,vinaco21,Watching a game,1,0,HUGE TRADE!\n\nhttps://t.co/1a9TcuqqOM,[],[],Flyers,recent,positive,-0.7717,-0.6203
529,MPence93,"Harrisonburg, VA",0,0,"@KarlJacobs_ @MrBeast @Twitch They begged, but...",[],[],Twitch,recent,positive,-0.7644,-0.6266
355,fatima_bhaiyat,glos,0,0,Prince Harry is on the right side of History… ...,[],[],Prince Harry,recent,positive,-0.7638,-0.6271
146,Debbiesm2022,Missouri,0,0,@Jim_Jordan Still don't know about you after t...,[],[],Jim Jordan,recent,positive,-0.7581,-0.6321
2465,tkn_gmr,USA,91,550,R E S E T ‼️‼️‼️‼️\n\n🇵🇰@ArslanAsh95\n🇰🇷@holyk...,"[{'text': 'TWT2023', 'indices': [75, 83]}, {'t...","[🇵🇰, ‼️, 🇰🇷]",Twitch,popular,positive,-0.7569,-0.6332
597,SadiqBalah,,0,0,"Saudi please sign me, na beg i beg 🙏🏾 https://...",[],[🙏],Saudi,recent,positive,-0.7567,-0.6334
438,Ribonucleotidic,"Los Angeles, 2019",3,0,This is a joke. 😄 #UFOs https://t.co/JyOcnORaoQ,"[{'text': 'UFOs', 'indices': [38, 43]}]",[😄],UFOs,recent,positive,-0.7557,-0.6343
2577,NintendoAmerica,,10648,87529,Kick off your summer in style with this new li...,"[{'text': 'Nintendo', 'indices': [122, 131]}]",[],Kick,popular,positive,-0.7542,-0.6356
2971,dltnio,,253,2690,@Daniel_Rapaport rory https://t.co/0rRqPESHTw,[],[],Rory,popular,positive,-0.7529,-0.6368


In [68]:
pred_negative_tweets = tweets_data.sort_values(by=['neg_prob'], inplace = False, ascending = False)        
                            
top_negative = pred_negative_tweets[0:25]     

top_negative

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type,sentiment,neg_prob,pos_prob
2036,phil_heal,Chicago,0,0,Take my money !!! https://t.co/mZSTiBAnTN,[],[],Phil,recent,negative,-0.5544,-0.8543
75,spiralcrunch,,2,0,@peta Tails grow back. https://t.co/XlImpvhVf7,[],[],PETA,recent,negative,-0.5870,-0.8119
826,UnKunumi,,0,0,Alcaraz is a monster.,[],[],Alcaraz,recent,negative,-0.5895,-0.8088
2327,RepMTG,"Rome, GA",3985,23144,IMPEACH BIDEN\n\nIMPEACH MAYORKAS\n\nIMPEACH G...,[],[],Garland,popular,negative,-0.5910,-0.8070
671,ItsLuiisE,"Las Cruces, NM",81,0,@zachbussey sign up - https://t.co/LGVqVLs3jL,[],[],Kick,recent,negative,-0.5920,-0.8057
555,NotLukeOrAni,,0,0,@MrBeast @Twitch Twitch: https://t.co/CD0KIUe3eO,[],[],Twitch,recent,negative,-0.5957,-0.8011
1548,NoonkesterJoel,Texas,0,1,@gtconway3d Barr is a crooked RINO..,[],[],Barr,recent,negative,-0.6008,-0.7949
848,diyahlau,,0,0,Back to back aces Alcaraz!,[],[],Alcaraz,recent,negative,-0.6048,-0.7901
1482,CanadianFlyera,Canada,0,2,@hugodebellefeu2 @NHLFlyers give Briere a raise,[],[],Briere,recent,negative,-0.6064,-0.7882
2133,BuzzingPop,,249,2998,Ariana Grande’s @rembeauty is now officially P...,[],[],PETA,popular,negative,-0.6116,-0.7819
